 # this file will try to trian the two models defined here. And determine which is the best performing one

In [4]:
from torch.nn import Module, Sequential, LeakyReLU, Conv2d, BatchNorm2d, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Linear, Dropout
import torch
from torchinfo import summary
from customModelsInception import *

# this is adapted from https://github.com/Moeo3/GoogLeNet-Inception-V3-pytorch/blob/master/googlenet_v3.py#L58 with its size modified to suit the CIFAR10 dataset instead of the origianl ImageNet dataset.


Layer (type:depth-idx)                        Output Shape              Param #
InceptionModel3                               [64, 10]                  --
├─Sequential: 1-1                             [64, 32, 32, 32]          --
│    └─Conv2d_BN: 2-1                         [64, 32, 32, 32]          --
│    │    └─Sequential: 3-1                   [64, 32, 32, 32]          928
├─Sequential: 1-47                            --                        (recursive)
│    └─Conv2d_BN: 2-47                        --                        (recursive)
│    │    └─Sequential: 3-48                  --                        (recursive)
├─Sequential: 1-3                             [64, 64, 32, 32]          --
│    └─Conv2d_BN: 2-3                         [64, 64, 32, 32]          --
│    │    └─Sequential: 3-3                   [64, 64, 32, 32]          18,560
├─Sequential: 1-47                            --                        (recursive)
│    └─Conv2d_BN: 2-47                        --      

In [2]:
STUDENTID = 567     # this will be used for random states

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os
import time

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    # this normalization is industry standard, it seems
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the data
])


# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
    #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))

# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000


In [ ]:
model = InceptionModel3(channels_in=3, class_num=10)
modelName = 'InceptionModel3_base'

# this transformation sequence must be modified to work with the random stuff
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    # this normalization is industry standard, it seems
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the data
])

argDict = {
    'lr': 0.001,
    'maxEpoch': 250,
    'idleEpoch': 25,
    'outputName': modelName,
    'optimizer': optim.SGD(model.parameters(), lr=0.001),
    'criterion': nn.CrossEntropyLoss()
}

STUDENTID = 567     # this will be used for random states

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os
import time


# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
    #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))

# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

start_time = time.time()

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)

# timing the thing as well
end_time = time.time()
execution_time = end_time - start_time
argDict['time_taken'] = execution_time
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)